In [1]:
from models.TransposeTransformer import TransposeTransformer
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from utils.utils import train_model, evaluate_model_with_cm, TiffDataset, base_transform,he_init_weights
from models.video_classifier import VideoClassifier, BandExpansion, ResBlock, FeedForward,init
import os
import numpy as np
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
source_data_dir = rf"C:\Users\전승표\서울과기대\JupyterNotebook\Capstone\git_folder\data_final\source_data"
label_file_path = rf"C:\Users\전승표\서울과기대\JupyterNotebook\Capstone\git_folder\data_final\label_data\species\label_mapping_sampled.csv"
test_filter = lambda x: x >= 1 and (x % 50) in {1, 6, 13, 18, 25, 30, 32, 37, 44, 49}


In [2]:
epochs = 10
bands = 10
patch_size = 3 # 모델 안에서 3*3으로 변경

train_dataset = TiffDataset(
    large_tif_dir = os.path.join(source_data_dir,"with_s2_uint16_revised"),
    file_list = ["jiri_1.tif", "jiri_2.tif", "sobaek.tif"], #전체 지역을 모두 사용한다.
    label_file = label_file_path,
    box_filter_fn = lambda box_number: not test_filter(box_number),
    transform=base_transform(bands=bands, patch_size=patch_size),
    patch_size = patch_size
)
val_dataset = TiffDataset(
    large_tif_dir = os.path.join(source_data_dir,"with_s2_uint16_revised"),
    file_list = ["jiri_1.tif", "jiri_2.tif", "sobaek.tif"], #전체 지역을 모두 사용한다.
    label_file = label_file_path,
    box_filter_fn = lambda box_number: test_filter(box_number),
    transform=base_transform(bands=bands, patch_size=patch_size),
    patch_size = patch_size
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [ ]:
model = TransposeTransformer(num_blocks=4, 
                 num_head=10,
                 num_axis=12, 
                 num_dim=9,
                 num_classes=6, 
                 image_size=3,
                 patch_size=1, 
                 expand_ratio=2, 
                 droppath=0.2, 
                 dropout=0.1,device=device).to(device)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=5e-4, weight_decay=0.05)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, len(train_loader)*epochs)
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"파라미터 개수: {total_params}")
best_model_state, train_losses, val_losses = train_model(model, train_loader, val_loader, criterion, optimizer,scheduler=scheduler, num_epochs=epochs)

파라미터 개수: 456374


Epoch 1/10 - Training:   2%|▏         | 128/5709 [00:05<03:39, 25.46it/s]

# test

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from timm.layers import DropPath, trunc_normal_
import math
from models.TransposeTransformer import TransposeTransformer

import numpy as np
model = TransposeTransformer(num_blocks=2, 
                 num_head=10,
                 num_axis=12, 
                 num_dim=9,
                 num_classes=6, 
                 image_size=3,
                 patch_size=1, 
                 expand_ratio=2, 
                 droppath=0.0, 
                 dropout=0.0)

a = torch.tensor( np.arange(0,12*10*9) , dtype=torch.float32).view(1,10, 12, 3,3)
out = model(a)
print(out.shape, out)

time torch.Size([1, 10, 12, 9])
transformer time
band torch.Size([1, 12, 10, 9])
transformer band
spatial torch.Size([1, 12, 9, 10])
transformer spatial
time torch.Size([1, 10, 12, 9])
transformer time
band torch.Size([1, 12, 10, 9])
transformer band
spatial torch.Size([1, 12, 9, 10])
transformer spatial
torch.Size([1, 6]) tensor([[ 0.2583, -1.2302, -1.3778, -0.0181, -0.2155,  0.2224]],
       grad_fn=<AddmmBackward0>)


In [64]:
import numpy as np

In [81]:
a = torch.zeros(1, 2, 3, 3, 3) # batch, channel, time, height, width
a = torch.tensor( np.arange(0,2*27).reshape(2, 3, 3,3) ).unsqueeze(0)
model = Embedding()
out = model(a)#.permute(0, 1, 3 ,2).reshape(1, 5, 27)
#out = model(a).permute(0, 1, 2, 3 ).reshape(1, 5, 27)
print(out.shape, out)

torch.Size([1, 2, 3, 3, 3]) tensor([[[[[ 0,  1,  2],
           [ 3,  4,  5],
           [ 6,  7,  8]],

          [[ 9, 10, 11],
           [12, 13, 14],
           [15, 16, 17]],

          [[18, 19, 20],
           [21, 22, 23],
           [24, 25, 26]]],


         [[[27, 28, 29],
           [30, 31, 32],
           [33, 34, 35]],

          [[36, 37, 38],
           [39, 40, 41],
           [42, 43, 44]],

          [[45, 46, 47],
           [48, 49, 50],
           [51, 52, 53]]]]], dtype=torch.int32)


In [82]:
out = out.reshape(1, 2, 3* 9)
print(out.shape, out)
out = out.reshape(1, 2,3,9).permute(0,2,1,3).reshape(1,3,2*9) 
out = out.reshape(1, 3,2,9).permute(0,2,1,3).reshape(1, 2, 3*9)

print(out.shape, out)

torch.Size([1, 2, 27]) tensor([[[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
          17, 18, 19, 20, 21, 22, 23, 24, 25, 26],
         [27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
          44, 45, 46, 47, 48, 49, 50, 51, 52, 53]]], dtype=torch.int32)
torch.Size([1, 2, 27]) tensor([[[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
          17, 18, 19, 20, 21, 22, 23, 24, 25, 26],
         [27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
          44, 45, 46, 47, 48, 49, 50, 51, 52, 53]]], dtype=torch.int32)


In [ ]:
class Embedding(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        batch,  channel, time, height, width = x.size()
        basic = torch.ones( (height,1), dtype=torch.int16)
        basic2 = torch.ones( (width,1), dtype=torch.int16)
        basic2[width//2,0] = 0 
        position_embedding = basic@basic2.T + basic2
        position_embedding = torch.sqrt(position_embedding)*2.5
        x = x + position_embedding
        x = x.view(batch, channel, time, height * width) 
        
        return x

In [6]:
class Attention(nn.Module):
    def __init__(self, num_heads, num_axis, num_dim):
        super().__init__()
        self.num_heads = num_heads
        self.num_axis = num_axis
        self.num_dim = num_dim

        #self.w_q = nn.ModuleList([nn.Linear(self.num_dim, self.num_dim)
        #                          for _ in range(self.num_heads)])
        #self.w_k = nn.ModuleList([nn.Linear(self.num_dim, self.num_dim) 
        #                          for _ in range(self.num_heads)])
        #self.w_v = nn.ModuleList([nn.Linear(self.num_dim, self.num_dim) 
        #                          for _ in range(self.num_heads)])
        #self.w_o = nn.ModuleList([nn.Linear(self.num_dim, self.num_dim) 
        #                          for _ in range(self.num_heads)])
        self.w_q = nn.Linear(num_axis*self.num_dim, num_axis*self.num_dim)
        self.w_k = nn.Linear(num_axis*self.num_dim, num_axis*self.num_dim)
        self.w_v = nn.Linear(num_axis*self.num_dim, num_axis*self.num_dim)
        self.w_o = nn.Linear(num_axis*self.num_dim, num_axis*self.num_dim)

    def forward(self, x):
        
        #q_outputs = []
        #k_outputs = []
        #v_outputs = []
        #for i in range(self.num_heads):
        #    # 각 헤드별 선형 투영
        #    q_i = self.w_q[i](x[:,i,:,:]) 
        #    k_i = self.w_k[i](x[:,i,:,:]) 
        #    v_i = self.w_v[i](x[:,i,:,:]) 
        #    q_outputs.append(q_i)
        #    k_outputs.append(k_i)
        #    v_outputs.append(v_i)

        #q = torch.stack(q_outputs, dim=1)
        #k = torch.stack(k_outputs, dim=1)
        #v = torch.stack(v_outputs, dim=1)
        x = x.view(-1, self.num_heads, self.num_axis*self.num_dim)
        q = self.w_q(x).view(-1, self.num_heads, self.num_axis, self.num_dim)
        k = self.w_k(x).view(-1, self.num_heads, self.num_axis, self.num_dim)
        v = self.w_v(x).view(-1, self.num_heads, self.num_axis, self.num_dim)

        score = torch.matmul(q,k.transpose(-2,-1))
        score = score / math.sqrt(self.num_dim)
        attention_weights = F.softmax(score, dim=-1)
        context = torch.matmul(attention_weights, v)
        
        #for i in range(self.num_heads):
        #    context_i = self.w_o[i](context[:,i,:,:])
        #    context[:,i,:,:] = context_i
        context = context.view(-1, self.num_heads,self.num_axis*self.num_dim)
        context = self.w_o(context).view(-1, self.num_heads, self.num_axis, self.num_dim)
        return context

In [8]:
### attention input test
import math
a = torch.tensor( np.arange(0,2*27) , dtype=torch.float32).view(1,2, 3, 3,3)
out =a.view(1, 2, 3, 9) ##한 시점안의 관련공간에 관한 밴드 attention
print(out.shape, out)
model = Attention( 2, 3, 9 )
out = model(out)
print(out.shape, out)

torch.Size([1, 2, 3, 9]) tensor([[[[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12., 13., 14., 15., 16., 17.],
          [18., 19., 20., 21., 22., 23., 24., 25., 26.]],

         [[27., 28., 29., 30., 31., 32., 33., 34., 35.],
          [36., 37., 38., 39., 40., 41., 42., 43., 44.],
          [45., 46., 47., 48., 49., 50., 51., 52., 53.]]]])
torch.Size([1, 2, 3, 9]) tensor([[[[ -6.0152,   2.8868,   5.5611,  -1.2698,   5.3441,   7.3263,  -0.8581,
             2.0172,   2.5754],
          [  5.5988,  -3.8020,   0.3109,  -7.6027,   0.8487,  -3.8262,   3.7686,
            -3.9909,  -1.7513],
          [ -7.2704,  -6.0137,  -1.9150,   3.8115,   2.4304,   7.8921,   1.5275,
             0.9442,  -2.1012]],

         [[-13.8452,  11.3892,  16.3185,  -0.7192,  23.6106,  26.4914, -10.4970,
             9.6060,   3.5596],
          [  6.9093, -11.2839,  -7.1442, -24.7043,  -0.7202, -14.3800,  14.9439,
           -10.7903,  -4.1461],
          [-29.8874, -26.4242, -10.3

In [179]:
class FeedForward(nn.Module):
    def __init__(self, dim, expand_ratio=2):
        super().__init__()
        hidden_dim = dim * expand_ratio
        self.w1 = nn.Linear(dim, hidden_dim)
        self.w2 = nn.Linear(hidden_dim, dim)
    
    def forward(self, x):
        return self.w2(F.silu(self.w1(x)))

In [205]:
class TransposeTransformerBlock(nn.Module):
    def __init__(self,  num_heads, num_axis,num_dim, expand_ratio,tag, dropout=0., attn_drop_rate=0., droppath=0.):
        super().__init__()
        self.token_norm = nn.LayerNorm(num_dim)
        self.token_mixer = Attention(num_heads, num_axis, num_dim)
        self.channel_norm = nn.LayerNorm(num_dim)
        self.channel_mixer = FeedForward(num_dim, expand_ratio)
        self.droppath = DropPath(droppath) if droppath > 0. else nn.Identity()
        self.tag = tag
    def forward(self, x):
        print('transformer',self.tag)
        x = self.token_norm(x)
        x = self.token_mixer(x)
        x = x + self.droppath(x)
        x = x + self.droppath(self.channel_mixer(self.channel_norm(x)))
        return x


In [214]:
class TransposeTransformer(nn.Module):
    def __init__(self, 
                 num_blocks=3, 
                 num_head=10,
                 num_axis=12, 
                 num_dim=9,
                 num_classes=6, 
                 image_size=3,
                 patch_size=1, 
                 expand_ratio=2, 
                 droppath=0.0, 
                 dropout=0.0
        ):
        super().__init__()
        self.num_head = num_head
        self.num_axis = num_axis
        self.num_dim = num_dim
        self.num_classes = num_classes
        self.image_size = image_size
        self.num_blocks = num_blocks

        self.embedding_layer = Embedding()

        self.time_layers = nn.Sequential(*[
            TransposeTransformerBlock(self.num_head, self.num_axis, self.num_dim,
                                       expand_ratio,'time',dropout=dropout, droppath=droppath)  
            for _ in range(self.num_blocks)]
        )
        self.band_layers = nn.Sequential(*[
            TransposeTransformerBlock( self.num_axis, self.num_head, self.num_dim, 
                                      expand_ratio,'band',dropout=dropout, droppath=droppath)  
            for _ in range(self.num_blocks)]
        )
        self.spatial_layers = nn.Sequential(*[
            TransposeTransformerBlock( self.num_axis, self.num_dim, self.num_head, 
                                      expand_ratio,'spatial',dropout=dropout, droppath=droppath)  
            for _ in range(self.num_blocks)]
        )

        self.dropout = nn.Dropout(dropout)
        self.norm = nn.LayerNorm(self.num_head* self.num_axis)
        self.head = nn.Linear( self.num_head* self.num_axis , num_classes)
    
    def forward(self, x):
        x = self.embedding_layer(x)

        for idx in range(self.num_blocks):
            print('time',x.shape)
            x = self.time_layers[idx](x)
            x = x.permute(0,2,1,3)
            print('band',x.shape)
            x = self.band_layers[idx](x)
            x = x.permute(0,1,3,2)
            print('spatial',x.shape)
            x = self.spatial_layers[idx](x)
            x = x.permute(0,3,1,2)
        x = x.mean([3]).view(-1,self.num_head* self.num_axis )
        x = self.norm(x)
        x = self.dropout(x)
        x = self.head(x)
        return x

In [215]:
model = TransposeTransformer(num_blocks=2, 
                 num_head=10,
                 num_axis=12, 
                 num_dim=9,
                 num_classes=6, 
                 image_size=3,
                 patch_size=1, 
                 expand_ratio=2, 
                 droppath=0.0, 
                 dropout=0.0)

a = torch.tensor( np.arange(0,12*10*9) , dtype=torch.float32).view(1,10, 12, 3,3)
out = model(a)
print(out.shape, out)

time torch.Size([1, 10, 12, 9])
transformer time
band torch.Size([1, 12, 10, 9])
transformer band
spatial torch.Size([1, 12, 9, 10])
transformer spatial
time torch.Size([1, 10, 12, 9])
transformer time
band torch.Size([1, 12, 10, 9])
transformer band
spatial torch.Size([1, 12, 9, 10])
transformer spatial
torch.Size([1, 6]) tensor([[ 1.3707, -0.1266, -0.2153, -0.9284, -0.3243, -0.6217]],
       grad_fn=<AddmmBackward0>)


In [213]:
a = torch.tensor( np.arange(0,2*27) , dtype=torch.float32).view(1,2, 3, 3,3)
out = a
out =out.view(1, 2, 27) 
print(out.shape, out)
out = out.view(1, 2, self.num_heads, self.split_dim)
print(out.shape, out)

torch.Size([1, 2, 27]) tensor([[[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
          14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.],
         [27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39., 40.,
          41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51., 52., 53.]]])


NameError: name 'self' is not defined

In [173]:
### transformer input shape test
a = torch.tensor( np.arange(0,2*27) , dtype=torch.float32).view(1,2, 3, 3,3)
model = Embedding()
out = model(a)
#print(out.shape, out)
##시점 간의 관련공간에 관한 밴드 attention => head=image_size
##밴드 간의 관련공간에 관한 시점 attention => head=time_size
##픽셀 간의 관련 밴드와 시점 attention => head= 줘야 될것 같다?애매.
out =out.view(1, 2, 3, 9) 
print(out.shape, out)
print(out.mean([1]).shape,out.mean([1]))
print(out.mean([2]).shape,out.mean([2]))
print(out.mean([3]).shape,out.mean([3]))
out = out.permute(0,2,1,3).contiguous()
#print(out.shape, out)
out = out.permute(0,1,3,2).contiguous()
#print(out.shape, out)
out = out.permute(0,3,1,2).contiguous()
#print(out.shape, out)
out = out.permute(0,2,1,3).contiguous()
#print(out.shape, out)

torch.Size([1, 2, 3, 9]) tensor([[[[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12., 13., 14., 15., 16., 17.],
          [18., 19., 20., 21., 22., 23., 24., 25., 26.]],

         [[27., 28., 29., 30., 31., 32., 33., 34., 35.],
          [36., 37., 38., 39., 40., 41., 42., 43., 44.],
          [45., 46., 47., 48., 49., 50., 51., 52., 53.]]]])
torch.Size([1, 3, 9]) tensor([[[13.5000, 14.5000, 15.5000, 16.5000, 17.5000, 18.5000, 19.5000,
          20.5000, 21.5000],
         [22.5000, 23.5000, 24.5000, 25.5000, 26.5000, 27.5000, 28.5000,
          29.5000, 30.5000],
         [31.5000, 32.5000, 33.5000, 34.5000, 35.5000, 36.5000, 37.5000,
          38.5000, 39.5000]]])
torch.Size([1, 2, 9]) tensor([[[ 9., 10., 11., 12., 13., 14., 15., 16., 17.],
         [36., 37., 38., 39., 40., 41., 42., 43., 44.]]])
torch.Size([1, 2, 3]) tensor([[[ 4., 13., 22.],
         [31., 40., 49.]]])


In [ ]:
### transformer input shape test
a = torch.tensor( np.arange(0,2*27) , dtype=torch.float32).view(1,2, 3, 3,3)
model = Embedding()
out = model(a)
print(a.shape, a)
##시점 간의 관련공간에 관한 밴드 attention => head=image_size
out =out.view(1, 2, 27) 
print(out.shape, out)
##밴드 간의 관련공간에 관한 시점 attention => head=time_size
out = out.view(1, 2, 3, 9).permute(0,2,1,3).contiguous().view(1,3,2*9) 
print(out.shape, out)
##픽셀 간의 관련 밴드와 시점 attention => head= 줘야 될것 같다?애매.
out = out.view(1, 3, 2,9 ).permute(0,3,2,1).contiguous().view(1,9,2*3) 
print(out.shape, out)

torch.Size([1, 2, 3, 9]) tensor([[[[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12., 13., 14., 15., 16., 17.],
          [18., 19., 20., 21., 22., 23., 24., 25., 26.]],

         [[27., 28., 29., 30., 31., 32., 33., 34., 35.],
          [36., 37., 38., 39., 40., 41., 42., 43., 44.],
          [45., 46., 47., 48., 49., 50., 51., 52., 53.]]]])
torch.Size([1, 2, 27]) tensor([[[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
          14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.],
         [27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39., 40.,
          41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51., 52., 53.]]])
torch.Size([1, 3, 18]) tensor([[[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8., 27., 28., 29., 30., 31.,
          32., 33., 34., 35.],
         [ 9., 10., 11., 12., 13., 14., 15., 16., 17., 36., 37., 38., 39., 40.,
          41., 42., 43., 44.],
         [18., 19., 20., 21., 22., 23., 24., 2